In [21]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from PIL import Image

In [22]:
sys.path.append('../../codes')
from utils import video_utils

In [23]:
!mkdir "../../Regioes/tubiacanga"
!mkdir "../../Regioes/tubiacanga/frames"
!mkdir "../../Regioes/tubiacanga/frames/all"
!mkdir "../../Regioes/tubiacanga/frames/overlaped"
!mkdir "../../Regioes/tubiacanga/dataFrames"
!mkdir "../../Regioes/tubiacanga/dataFrames/all"
!mkdir "../../Regioes/tubiacanga/dataFrames/overlaped"


J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga.
J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga/frames.
J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga/frames/all.


J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga/frames/overlaped.
J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga/dataFrames.
J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga/dataFrames/all.
J� existe uma subpasta ou um arquivo ../../Regioes/tubiacanga/dataFrames/overlaped.


In [24]:
ALL_FRAMES_STRING = 'tubiacanga/frames/all/'
OVERLAPED = '../../Regioes/tubiacanga/dataFrames/overlaped/'

In [25]:
VIDEO_PATH = '../../Dataset/videos/tubiacanga/'
VIDEO_NAMES = ['20190601_rectfied_DJI_0002.avi','20190601_rectfied_DJI_0003.avi']


In [26]:
videos = []
logs = []
for i in VIDEO_NAMES:
  videos.append(VIDEO_PATH + i)

In [27]:
videos

['../../Dataset/videos/tubiacanga/20190601_rectfied_DJI_0002.avi',
 '../../Dataset/videos/tubiacanga/20190601_rectfied_DJI_0003.avi']

In [28]:
videos_obj = []
videos_capture = []
for i in videos:
  videos_capture.append(cv2.VideoCapture(i))

## Logging phase_correlation

In [ ]:
frames_count = 0
for path in os.listdir(all_path):
  if os.path.isfile(os.path.join(all_path, path)):
    frames_count += 1
r_array = []
x = []
y = []
c = []
counter = 1
frameB = cv2.imread(all_path + 'frame_{}.jpeg'.format('{:05d}').format(counter))
while counter < frames_count:
  counter += 1
  frameA = frameB
  frameB = cv2.imread(all_path + 'frame_{}.jpeg'.format('{:05d}').format(counter))
  results = phase_correlation(frameA, frameB, scale = 0.4)
  r_array.append(results)
  x.append(results[0][0])
  y.append(results[0][1])
  c.append(results[1])
print (len(r_array))
x_dataFrame = pd.DataFrame(x)
x_dataFrame.columns = ['x']
y_dataFrame = pd.DataFrame(y)
y_dataFrame.columns = ['y']
c_dataFrame = pd.DataFrame(c)
c_dataFrame.columns = ['c']
r_dataFrame = pd.concat([x_dataFrame, y_dataFrame, c_dataFrame], axis=1)
return r_dataFrame

In [32]:
def phase_correlation(img, img_offset, scale=None):
    """Image translation registration by cross-correlation.
    It obtains an estimate of the cross-correlation peak by an FFT.
    It is very similar to `skimage.feature.register_translation`.
    However, our function runs faster because we restrict it to our application.
    Args:
        img (array): image.
        img_offset (array): offset image. Must have the same dim as `img`.
        scale (float, optional): If not `None`, rescale input images to run faster.
        Defaults to None.
    Returns:
        array:  shift vector (in pixels) required to register `img_offset` with
        `img`.  Axis ordering is consistent with numpy (e.g. Z, Y, X)
    """

    if img.shape != img_offset.shape:
        raise ValueError("Error: images must be same size")

    # if images in BGR, tranform to grayscal and use fft2
    # which is much faster than using fftn
    # if img.ndim > 2:
    #     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # if img_offset.ndim > 2:
    #     img_offset = cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY)

    img = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    img_offset = np.float32(cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY))

    if scale is not None:
        img = scale_img(img, scale)
        img_offset = scale_img(img_offset, scale)

    (x, y), c = cv2.phaseCorrelate(img, img_offset)

    # img_fft = np.fft.fft2(img)
    # img_offset_fft = np.fft.fft2(img_offset)

    # shape = img_fft.shape
    # midpoints = np.array([np.fix(axis_size / 2) for axis_size in shape])

    # R = img_fft * img_offset_fft.conj()
    # # R /= np.absolute(R)  # normalize give wrong results for large frames interval (???)
    # # (print('norm'))
    # cross_correlation = np.fft.ifft2(R)

    # shifts = np.unravel_index(np.argmax(np.absolute(cross_correlation)), shape)
    # shifts = np.array(shifts, dtype=np.float64)
    # shifts[shifts > midpoints] -= np.array(shape)[shifts > midpoints]

    if scale is not None:
        x /= scale
        y /= scale

    return (x, y), c

In [33]:
r_array = []
x = []
y = []
c = []
counter = 0
for i in videos_capture:
    frameB = i.read()
    while frameB[0]:
        counter += 1
        frameA = frameB
        frameB = i.read()
        results = phase_correlation(frameA[1], frameB[1], 0.4)
        r_array.append(results)
        x.append(results[0][0])
        y.append(results[0][1])
        c.append(results[1])
print(frameA)


KeyboardInterrupt: 

## Esquecido por falta de memória e velocidade

In [ ]:
counter = 0
total_frames = []
for i in videos_obj:
  total_frames.append(i.videoInfo.getNumberOfFrames())
for i in range(len(videos_capture)):
  aux_counter = 0
  while aux_counter < total_frames[i]:
    counter = counter + 1
    aux_counter = aux_counter + 1
    frame = videos_capture[i].read()
    image = Image.fromarray(cv2.cvtColor(frame[1],cv2.COLOR_BGR2RGB))
    string = ALL_FRAMES_STRING + 'frame_{}.jpeg'.format('{:05d}'.format(counter))
    image.save(string,'JPEG')